In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

# 추가 임포트
import pickle
from PIL import Image
import os
import nltk
from pycocotools.coco import COCO
import torch.utils.data as data
import torchvision.models as model
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence # 다양한 길이의 문장에 패딩을 적용해서 고정길이로 변환

In [2]:
from collections import Counter

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 이미지 캡션 데이터셋 다운로드

1. Training은 13GB, Validation은 6GB
2. 밤새 코드를 실행해놓으라고?
3. https://cocodataset.org/#home 가 출처 

In [4]:
!ls "./drive/MyDrive/Colab Notebooks/AI_study"
# !mkdir "dataset"
!cd "dataset"

2장_파이토치_맛보기.ipynb	 mnist_pytorch_practice.ipynb
for_git_commit.ipynb		 파이토치교과서_2장_연습.ipynb
image_captioning_practice.ipynb
/bin/bash: line 0: cd: dataset: No such file or directory


In [5]:
import urllib
import zipfile
from tqdm import tqdm

#https://stackoverflow.com/a/53877507/1558946
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_data(url):
    print(f"{url} 다운로드 중 ...")
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        zip_path, _ = urllib.request.urlretrieve(url, reporthook=t.update_to)

    print("압축을 푸는 중 ...")
    with zipfile.ZipFile(zip_path, "r") as f:
        for name in tqdm(iterable=f.namelist(), total=len(f.namelist())):
            f.extract(member=name, path="data_dir")


# download_data("http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip")
download_data("http://images.cocodataset.org/zips/train2014.zip")
download_data("http://images.cocodataset.org/zips/val2014.zip")

http://images.cocodataset.org/zips/train2014.zip 다운로드 중 ...


train2014.zip: 13.5GB [06:38, 33.9MB/s]                            


압축을 푸는 중 ...


100%|██████████| 82784/82784 [02:07<00:00, 647.46it/s]


http://images.cocodataset.org/zips/val2014.zip 다운로드 중 ...


val2014.zip: 6.65GB [03:17, 33.7MB/s]                            


압축을 푸는 중 ...


100%|██████████| 40505/40505 [01:00<00:00, 664.61it/s]


In [6]:
download_data("http://images.cocodataset.org/annotations/annotations_trainval2014.zip")

http://images.cocodataset.org/annotations/annotations_trainval2014.zip 다운로드 중 ...


annotations_trainval2014.zip: 253MB [00:08, 31.0MB/s]                           


압축을 푸는 중 ...


100%|██████████| 6/6 [00:08<00:00,  1.45s/it]


In [7]:
class Vocab(object):
    def __init__(self):
        self.w2i = {} # word to index
        self.i2w = {} # index to word
        self.index = 0 # adding value
    def __call__(self, token):
        if not token in self.w2i:
            return self.w2i['<unk>']
        return self.w2i[token]
    def __len__(self):
        return len(self.w2i)
    def add_token(self, token):
        if not token in self.w2i:
            self.w2i[token] = self.index
            self.i2w[self.index] = token
            self.index += 1

실제 텍스트 토큰을 숫자 토큰으로 전환할 수 있는 사전 구축.

In [8]:
def build_vocabulary(json, threshold):
    # 사전 wrapper 구축
    coco = COCO(json)
    counter = Counter()
    ids = coco.anns.keys()
    for i, id in enumerate(ids):
        caption = str(coco.anns[id]['caption'])
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        counter.update(tokens)
        if (i+1) % 1000 == 0:
            print("[{}/{}] Tokenized the captions.".format(i+1, len(ids)))
    tokens = [token for token, cnt in counter.items()]
    vocab = Vocab()
    vocab.add_token('<pad>')
    vocab.add_token('<start>')
    vocab.add_token('<end>')
    vocab.add_token('<unk>')

    for i, token in enumerate(tokens):
        vocab.add_token(token)
    return vocab

In [9]:
vocab = build_vocabulary(json = 'data_dir/annotations/captions_train2014.json',
                         threshold = 4)
vocab_path = './data_dir/vocabulary.pkl'
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)
print("Total vocabulary size: {}".format(len(vocab)))
print("Saved the vocabulary wrapper to '{}'".format(vocab_path))

loading annotations into memory...
Done (t=0.78s)
creating index...
index created!
[1000/414113] Tokenized the captions.
[2000/414113] Tokenized the captions.
[3000/414113] Tokenized the captions.
[4000/414113] Tokenized the captions.
[5000/414113] Tokenized the captions.
[6000/414113] Tokenized the captions.
[7000/414113] Tokenized the captions.
[8000/414113] Tokenized the captions.
[9000/414113] Tokenized the captions.
[10000/414113] Tokenized the captions.
[11000/414113] Tokenized the captions.
[12000/414113] Tokenized the captions.
[13000/414113] Tokenized the captions.
[14000/414113] Tokenized the captions.
[15000/414113] Tokenized the captions.
[16000/414113] Tokenized the captions.
[17000/414113] Tokenized the captions.
[18000/414113] Tokenized the captions.
[19000/414113] Tokenized the captions.
[20000/414113] Tokenized the captions.
[21000/414113] Tokenized the captions.
[22000/414113] Tokenized the captions.
[23000/414113] Tokenized the captions.
[24000/414113] Tokenized the 

In [10]:
!mkdir data_dir/resized_images

In [11]:
def reshape_image(image, shape):
    return image.resize(shape, Image.ANTIALIAS)
def reshape_images(image_path, output_path, shape):
    images = os.listdir(image_path)
    num_im = len(images)
    for i, im in enumerate(images):
        with open(os.path.join(image_path, im), 'r+b') as f:
            with Image.open(f) as image:
                image = reshape_image(image, shape)
                image.save(os.path.join(output_path, im), image.format)
            
        if (i+1) % 100 == 0:
            print("[{}/{}] Resized the images and saved into '{}'.".format(i+1, num_im, output_path))
image_path = './data_dir/train2014/'
output_path = './data_dir/resized_images/'
image_shape = [256, 256]
reshape_images(image_path, output_path, image_shape)


[100/82783] Resized the images and saved into './data_dir/resized_images/'.
[200/82783] Resized the images and saved into './data_dir/resized_images/'.
[300/82783] Resized the images and saved into './data_dir/resized_images/'.
[400/82783] Resized the images and saved into './data_dir/resized_images/'.
[500/82783] Resized the images and saved into './data_dir/resized_images/'.
[600/82783] Resized the images and saved into './data_dir/resized_images/'.
[700/82783] Resized the images and saved into './data_dir/resized_images/'.
[800/82783] Resized the images and saved into './data_dir/resized_images/'.
[900/82783] Resized the images and saved into './data_dir/resized_images/'.
[1000/82783] Resized the images and saved into './data_dir/resized_images/'.
[1100/82783] Resized the images and saved into './data_dir/resized_images/'.
[1200/82783] Resized the images and saved into './data_dir/resized_images/'.
[1300/82783] Resized the images and saved into './data_dir/resized_images/'.
[1400/82

In [12]:
class CustomCocoDataset(data.Dataset):
    def __init__(self, data_path, coco_json_path, vocabulary, transform=None):
        self.root = data_path
        self.coco_data = COCO(coco_json_path)
        self.indices = list(self.coco_data.anns.keys())
        self.vocabulary = vocabulary
        self.transform = transform

    def __getitem__(self, idx):
        coco_data = self.coco_data
        vocabulary = self.vocabulary
        annotation_id = self.indices[idx]
        caption = coco_data.anns[annotation_id]['caption']
        image_id = coco_data.anns[annotation_id]['image_id']
        image_path = coco_data.loadImgs(image_id)[0]['file_name']

        image = Image.open(os.path.join(self.root, image_path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        
        word_tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(vocabulary('<start>'))
        caption.append([vocabulary(token) for token in word_tokens])
        caption.append(vocabulary('<end>'))
        ground_truth = torch.Tensor(caption)

        return image, ground_truth

    def __len__(self):
        return len(self.indices)

